In [1]:
import sys
from bs4 import BeautifulSoup
import requests
import datetime as dt
from PIL import Image
import helpers as hf

import yaml
from unidecode import unidecode
import pandas as pd
from PIL import Image, ImageOps, ImageDraw, ImageFont
import json
import numpy as np


with open("info/design_config.yml", "r") as f:
    design_config = yaml.safe_load(f)

design_config,shop_config = hf.get_config('cbb')
cfbd_loader = hf.cfbp_handler(design_config['cfbd_api'])

private file size: 2466


In [2]:
teams = cfbd_loader.get_team_info()

81.6

In [3]:
with open("info/2024_qualifiers.yml","r") as f:
    quals = yaml.safe_load(f)

In [4]:
# quals=list(set(quals))
# with open('info/2024_qualifiers.yml', 'w') as file:
#     yaml.dump(quals, file)


In [5]:
teamnames=[x.lower() for x in teams.name]


In [6]:
teams["joiner"] = teams.name.apply(lambda x: unidecode(x).lower())


In [7]:
for t in quals:
    team_joiner = unidecode(t).lower()
    team_info = teams.loc[teams.joiner==team_joiner]
    if len(team_info)==0:
        print(f'Missing {t}')
    elif (type(team_info.logos)==float) or (team_info.mascot is None):
        print(f"incomplete {f}")
    if len(team_info)>1:
        print(f"Dupe Info {t}")
    
  

Dupe Info South Florida


In [13]:
teams.loc[teams.joiner=='south florida']

,name,abrev,alt_color,color,division,id,logos,long_name,mascot,team_id,joiner
1424,South Florida,USF,#231f20,#004A36,fbs,58.0,http://a.espncdn.com/i/teamlogos/ncaa/500/58.png,None,Bulls,NaN,south florida


In [9]:
removers = [3198]

In [10]:
teams = teams.loc[teams.id.isin(removers)==False]

In [11]:
teams.joiner.value_counts()[0:20]

joiner
blinn college          4
independence c.c.      3
butler c.c.            3
montana tech           2
texas a&m commerce     2
west virginia state    2
georgetown ky          2
professional           2
texas wesleyan         2
hutchinson c.c.        2
edward waters          2
stillman               2
erskine                2
northwestern ia        2
la salle               2
american               2
pacific fleet          1
pace                   1
ouachita baptist       1
parsons                1
Name: count, dtype: int64

In [12]:
# add Oakland, UNC Wilmington, Sam Housten

In [286]:
ify_user = hf.shopify_printify(shop_config,'cbb')

In [287]:
ap = ify_user.recursive_get_prods()

In [205]:
# check if printify ptoduct is posted
headers_printify = {
    "Authorization": f"Bearer {shop_config['printify_access']}",
    "Content-Type": "application/json",
}

In [283]:
url =f"{shop_config['base_url']}/shops/{shop_config['cbb']['shop_id']}/products/65d2945e767be705420225c9.json"
response = requests.get(url=url, headers=headers_printify)

In [284]:
json.loads(response.text).keys()

dict_keys(['id', 'title', 'description', 'tags', 'options', 'variants', 'images', 'created_at', 'updated_at', 'visible', 'is_locked', 'external', 'blueprint_id', 'user_id', 'shop_id', 'print_provider_id', 'print_areas', 'print_details', 'sales_channel_properties', 'is_printify_express_eligible', 'is_printify_express_enabled'])

In [285]:
# check the last product posted with this, and then wait like 5 extra minutes and then should be good to do images. 
# also could prob get ID we need to update this way instead of relying on alt text
if 'external' in json.loads(response.text).keys() and json.loads(response.text)['external']['id'] != '':
    

{'id': '8367777775837',
 'handle': 'https://c2b32f-2.myshopify.com/products/usc-vs-vcu-16-mar-2011',
 'shipping_template_id': '95423561949'}